In [16]:
import os
import re
import gensim
import pymorphy2

In [2]:
model = gensim.models.KeyedVectors.load_word2vec_format("ruwikiruscorpora_0_300_20.bin.gz", binary=True)
model.init_sims(replace=True)

In [17]:
morph = pymorphy2.MorphAnalyzer()
punct = re.compile('^(.*?)([а-яА-ЯёЁ-]+)(.*?)$')
capit = re.compile('^[А-Я]+$')

In [18]:
pth_source = 'books_before/'
lst = os.listdir(pth_source)

pth_result = 'books_after/'

In [19]:
cotags = {'ADJF':'ADJ', # pymorphy2: word2vec 
'ADJS' : 'ADJ', 
'ADVB' : 'ADV', 
'COMP' : 'ADV', 
'GRND' : 'VERB', 
'INFN' : 'VERB', 
'NOUN' : 'NOUN', 
'PRED' : 'ADV', 
'PRTF' : 'ADJ', 
'PRTS' : 'VERB', 
'VERB' : 'VERB'}

In [20]:
#print(ord('А'))
#print(ord('Я'))
#print(ord('Ё'))
capit_letters = [chr(x) for x in range(1040,1072)] + ['Ё']
#print(capit_letters)

#### todo

* ~~capitalize~~
* ~~pos detection~~
* ~~cashing of word2vec queries~~
* ~~1st form extraction from pymorphy2 parse of word2vec response~~
* ~~pos matching for most similar words~~
* names detection and excluding from the process

In [22]:
def search_neighbour(word, pos):
    lex = word + '_' + cotags[pos]
    if lex in model:
        neighbs = model.most_similar([lex])
        for nei in neighbs:
            lex_n, ps_n = nei[0].split('_')
            if pos == ps_n:
                return lex_n
    return None

In [26]:
def flection(lex_neighb, tags):
    tags = frozenset(tags)
    prep_for_gen = morph.parse(lex_neighb)
    for ana in prep_for_gen:
        if ana.normal_form == lex_neighb:
            flect = ana.inflect(tags)
            word_to_replace = flect.word
            return word_to_replace
    return None   

In [33]:
model.most_similar(['холодный_ADJ'])

[('теплый_ADJ', 0.6912842392921448),
 ('холод_NOUN', 0.6222048997879028),
 ('прохладный_ADJ', 0.6178681254386902),
 ('влажный_ADJ', 0.6009092926979065),
 ('жаркий_ADJ', 0.5878441333770752),
 ('горячий_ADJ', 0.5811808109283447),
 ('ледяной_ADJ', 0.5738131999969482),
 ('сухой_ADJ', 0.5681436657905579),
 ('сырой_ADJ', 0.5447033643722534),
 ('холодно_ADV', 0.5389620661735535)]

In [27]:
cash_neighb = {}

for fl in lst:
    print (fl)
    i = 0
    f = open(pth_source + fl, 'r', encoding='utf-8')
    fw = open(pth_result + fl, 'w', encoding='utf-8')
    for line in f:
        new_line = []
        i += 1
        if i > 20:
            break
        line = line.strip()
        words = line.split(' ')
        for word in words:
            struct = punct.findall(word)
            if struct:
                struct = struct[0]
            else:
                new_line.append(word)
                continue
            #print (struct)
            wordform = struct[1]
            if wordform:
                if capit.search(wordform):
                    new_line.append(word)
                    continue
                else:
                    if wordform[0] in capit_letters:
                        capit_flag = 1
                    else:
                        capit_flag = 0
                parse_result = morph.parse(wordform)[0]
                for tg in cotags:
                    if tg in parse_result.tag:
                        lex = parse_result.normal_form
                        pos_tag = parse_result.tag.POS
                        if (lex, pos_tag) in cash_neighb:
                            lex_neighb = cash_neighb[(lex, pos_tag)]
                        else:
                            lex_neighb = search_neighbour(lex, pos_tag)
                            cash_neighb[(lex, pos_tag)] = lex_neighb
                        if not lex_neighb:
                            new_line.append(word)
                            break
                        else:
                            if pos_tag == 'NOUN':
                                if parse_result.tag.case == 'nomn' and parse_result.tag.number == 'sing':
                                    if capit_flag == 1:
                                        lex_neighb = lex_neighb.capitalize()
                                    new_line.append(struct[0] + lex_neighb + struct[2])
                                else:
                                    word_to_replace = flection(lex_neighb, parse_result.tag)
                                    if word_to_replace:
                                        if capit_flag == 1:
                                            word_to_replace = word_to_replace.capitalize()
                                        new_line.append(struct[0] + word_to_replace + struct[2])
                                    else:
                                        new_line.append(word)
                                    
                            elif pos_tag == 'ADJF':
                                if parse_result.tag.case == 'nomn' and parse_result.tag.number == 'sing':
                                    if capit_flag == 1:
                                        lex_neighb = lex_neighb.capitalize()
                                    new_line.append(struct[0] + lex_neighb + struct[2])
                                else:
                                    word_to_replace = flection(lex_neighb, parse_result.tag)
                                    if word_to_replace:
                                        if capit_flag == 1:
                                            word_to_replace = word_to_replace.capitalize()
                                        new_line.append(struct[0] + word_to_replace + struct[2])
                                    else:
                                        new_line.append(word)
                            
                            elif pos_tag == 'INFN':
                                if capit_flag == 1:
                                    lex_neighb = lex_neighb.capitalize()
                                new_line.append(struct[0] + lex_neighb + struct[2])
                            
                            elif pos_tag in ['ADVB', 'COMP', 'PRED']:
                                if capit_flag == 1:
                                    lex_neighb = lex_neighb.capitalize()
                                new_line.append(struct[0] + lex_neighb + struct[2])
                                
                            else:
                                word_to_replace = flection(lex_neighb, parse_result.tag)
                                if word_to_replace:
                                    if capit_flag == 1:
                                        word_to_replace = word_to_replace.capitalize()
                                    new_line.append(struct[0] + word_to_replace + struct[2])
                                else:
                                    new_line.append(word)
                        break
                    else:
                        new_line.append(word)
            else:
                new_line.append(''.join(struct))
        line_replace = ' '.join(new_line)
        fw.write(line_replace + '\n')
    f.close()
    fw.close()
            
            

        

FathersAndSons.txt


TypeError: 'OpencorporaTag' object is not iterable